In [1]:
!pip install --upgrade tdmclient


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [4]:
from tdmclient import ClientAsync

try:
    client = ClientAsync()
    node = client.wait_for_node()
    print("Connected to Thymio II")

except Exception as e:
    print("Error connecting to Thymio II:", e)

Connected to Thymio II


In [60]:
# Function to locally avoid obstacles, memory implemented
%%run_python

speed0 = 200      # nominal speed
speedGain = 2     # gain used with ground gradient
obstSpeedGain = [6, 4, -2, -4, -10]    # /100
                  # gains used with front proximity sensors 0..4

# Obstacle memory parameters
obstacle_memory = [0, 0, 0, 0, 0]  # memory timer for each sensor
MEMORY_TIME = 200      # cycles to keep avoiding after detection (N * 10ms)
DETECTION_THRESHOLD = 1500  # sensor value to trigger memory

timer_period[0] = 10  # 10ms sampling time

toggle = False

@onevent
def button_center():
    global speed0, obstacle_memory, toggle, speedGain, obstSpeedGain

    if button_center == 1:
        toggle = 1 if toggle == 0 else 0

@onevent
def timer0():
    global prox_ground_delta, prox_horizontal, speed0, speedGain, obstSpeedGain, motor_left_target, motor_right_target, obstacle_memory, MEMORY_TIME, DETECTION_THRESHOLD, toggle

    # acquisition from ground sensor
    diffDelta = prox_ground_delta[1] - prox_ground_delta[0]

    if toggle:
        spLeft = speed0 - speedGain * diffDelta
        spRight = speed0 + speedGain * diffDelta

        # update obstacle memory and apply avoidance
        for i in range(5):
            # Update memory: refresh if obstacle detected, otherwise decay
            if prox_horizontal[i] > DETECTION_THRESHOLD:
                obstacle_memory[i] = MEMORY_TIME
            elif obstacle_memory[i] > 0:
                obstacle_memory[i] -= 1

            # Apply current sensor reading
            spLeft += prox_horizontal[i] * obstSpeedGain[i] // 100
            spRight += prox_horizontal[i] * obstSpeedGain[4 - i] // 100

            # Apply memory-based avoidance (using memory value as if it were a sensor reading)
            if obstacle_memory[i] > 0:
                spLeft += obstacle_memory[i] * obstSpeedGain[i] // 100
                spRight += obstacle_memory[i] * obstSpeedGain[4 - i] // 100

    else:
        spLeft = 0
        spRight = 0

    # motor control
    motor_left_target = spLeft
    motor_right_target = spRight


In [48]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.wait_for_variables()
for i in range(100):
    # You can also replace "prox.horizontal" by an other prox sensor:
    print(list(node["prox.horizontal"]))
    await client.sleep(0.2)

[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[1076, 0, 0, 0, 0, 0, 0]
[1386, 0, 0, 0, 0, 0, 0]
[1441, 0, 0, 0, 0, 0, 0]
[1538, 0, 0, 0, 0, 0, 0]
[1587, 0, 0, 0, 0, 0, 0]
[1588, 0, 0, 0, 0, 0, 0]
[1588, 0, 0, 0, 0, 0, 0]
[1588, 0, 0, 0, 0, 0, 0]
[1596, 0, 0, 0, 0, 0, 0]
[1644, 0, 0, 0, 0, 0, 0]
[1724, 0, 0, 0, 0, 0, 0]
[1788, 0, 0, 0, 0, 0, 0]
[1861, 0, 0, 0, 0, 0, 0]
[1957, 0, 0, 0, 0, 0, 0]
[2019, 0, 0, 0, 0, 0, 0]
[2019, 0, 0, 0, 0, 0, 0]
[2065, 0, 0, 0, 0, 0, 0]
[2098, 0, 0, 0, 0, 0, 0]
[2146, 0, 0, 0, 0, 0, 0]
[2188, 0, 0, 0, 0, 0, 0]
[2231, 0, 0, 0, 0, 0, 0]
[2257, 0, 0, 0, 0, 0, 0]
[2303, 0, 0, 0, 0, 0, 0]
[2369, 0, 0, 0, 0, 0, 0]
[2458, 0, 0, 0, 0, 0, 0]
[2512, 0, 0, 0, 0, 0, 0]
[2512, 0, 0, 0, 0, 0, 0]
[2550, 0, 0, 0, 0, 0, 0]
[2583, 0, 0, 0, 0, 0, 0]
[2570, 0, 0, 0, 0, 0, 0]
[2574, 0, 0, 0, 0, 0, 0]
[2574, 0, 0, 0, 0, 0, 0]
[2